In [ ]:
%%writefile app.py

# Initialize the Hugging Face Hub
hub_llm = HuggingFaceHub(
    huggingfacehub_api_token=os.environ['HUGGINGFACEHUB_API_TOKEN'],
    repo_id='meta-llama/Meta-Llama-3-8B-Instruct',
    model_kwargs={"temperature": 0.5, "max_length": 10000, "min_length": 5000}
)

# Streamlit file uploader
uploaded_files = st.file_uploader("Choose PDF files", type="pdf", accept_multiple_files=True)

if uploaded_files:
    documents = []
    for uploaded_file in uploaded_files:
        # Get the original filename
        original_filename = uploaded_file.name

        # Save the uploaded file with the original filename
        with open(original_filename, "wb") as f:
            f.write(uploaded_file.getbuffer())

        # Load the saved PDF file using PyPDFLoader
        loader = PyPDFLoader(original_filename)
        documents.extend(loader.load())

    # Split the loaded documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    texts = text_splitter.split_documents(documents)

    # Initialize the embeddings model
    model_name = "BAAI/bge-base-en"
    encode_kwargs = {'normalize_embeddings': True}
    model_norm = HuggingFaceBgeEmbeddings(
        model_name=model_name,
        encode_kwargs=encode_kwargs
    )

    # Create a vector store
    persist_directory = 'db'
    embedding = model_norm
    vectordb = Chroma.from_documents(documents=texts, embedding=embedding, persist_directory=persist_directory)
    retriever = vectordb.as_retriever(search_kwargs={"k": 5})

    def process_llm_response(llm_response):
    # Extract the generated answer from the response
    # Split the answer on the basis of 'Question:' or similar markers if necessary
      answer = llm_response['result'].strip()

    # Display the generated answer
      st.write(answer)

    # Display sources
      st.write('\nSources:')
      unique_sources = set()
      for source in llm_response["source_documents"]:
        if 'source' in source.metadata:
            unique_sources.add(source.metadata['source'])
        elif 'file_name' in source.metadata:
            unique_sources.add(source.metadata['file_name'])
        else:
            unique_sources.add("Unknown source")

    # Display each unique source
      if unique_sources:
        for source in unique_sources:
            st.write(source)
      else:
        st.write("None")




    B_INST, E_INST = "[INST]", "[/INST]"
    B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
    DEFAULT_SYSTEM_PROMPT = """\
    You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.
    If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""

    def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT):
        SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
        prompt_template = B_INST + SYSTEM_PROMPT + instruction + E_INST
        return prompt_template

    sys_prompt = """you are a hepful AI assistant who answers questions correctly and precisely with the help of the intstructions/ context given. You only gieve the answer and not the entire prompt as your llm response
    """
    instruction = """CONTEXT:/n/n {context}/n

    Question: {question}"""

    # Create the prompt template once
    prompt_template = get_prompt(instruction, sys_prompt)

    llama_prompt = PromptTemplate(
        template=prompt_template, input_variables=["context", "question"]
    )

    chain_type_kwargs = {"prompt": llama_prompt}

    # Create the QA chain
    qa_chain = RetrievalQA.from_chain_type(
        llm=hub_llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs=chain_type_kwargs,
        return_source_documents=True
    )


Overwriting app.py


In [ ]:
!wget -q -O - ipv4.icanhazip.com

34.106.169.182


In [ ]:
! streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.106.169.182:8501

  Stopping...
^C
